#**Operaciones de Machine Learning**
#**Proyecto 1**: Crear un ambiente de desarrollo de machine learning en el cual sea posible la ingesta, validaci´on y transformación de datos.
#**Grupo: 5**


Importar librerias

In [20]:
import os
import requests
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif
import numpy as np
import tfx
import tensorflow_transform as tft
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.components import CsvExampleGen,StatisticsGen,SchemaGen,ExampleValidator,Transform
from tfx.v1.components import ImportSchemaGen
import tensorflow_data_validation as tfdv
from tensorflow_metadata.proto.v0 import schema_pb2,anomalies_pb2
from google.protobuf import text_format
from tfx.orchestration.metadata import sqlite_metadata_connection_config
from ml_metadata.metadata_store import MetadataStore
from ml_metadata.proto import metadata_store_pb2
print('TF version:', tf.__version__)
print('TFDV version:', tfdv.version.__version__)

TF version: 2.16.2
TFDV version: 1.16.1


#Cargar Dataset

In [2]:
 ## download the dataset
 # Directory of the raw data files
 _data_root = '/data/covertype'
 # Path to the raw training data
 _data_filepath = os.path.join(_data_root, 'covertype_test.csv')
 # Download data
 os.makedirs(_data_root, exist_ok=True)
 if not os.path.isfile(_data_filepath):
     #https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/
     url = 'https://docs.google.com/uc?export= \
     download&confirm={{VALUE}}&id=1lVF1BCWLH4eXXV_YOJzjR7xZjj-wAGj9'
     r = requests.get(url, allow_redirects=True, stream=True)
     open(_data_filepath, 'wb').write(r.content)
# Cargar el dataset
df = pd.read_csv(_data_filepath)
df.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2991,119,7,67,11,1015,233,234,133,1570,Commanche,C7202,1
1,2876,3,18,485,71,2495,192,202,144,1557,Commanche,C7757,1
2,3171,315,2,277,9,4374,213,237,162,1052,Rawah,C7745,0
3,3087,342,13,190,31,4774,193,221,166,752,Rawah,C7745,0
4,2835,158,10,212,41,3596,231,242,141,3280,Rawah,C4744,1


#Selección de Caracteristicas

In [3]:
# Separar la variable objetivo
target = 'Cover_Type'
numerical_features = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Eliminar la variable objetivo de la lista de características
numerical_features.remove(target)

# Crear el subconjunto solo con las características numéricas
df_numerical = df[numerical_features + [target]]

print("Características numéricas:", numerical_features)

Características numéricas: ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points']


In [4]:

# Definir variables de entrada (X) y salida (y)
X = df_numerical.drop(columns=[target])
y = df_numerical[target]

k = 8  # Número de características a seleccionar
selector = SelectKBest(score_func=f_classif, k=k)
X_new = selector.fit_transform(X, y)

# Obtener las características seleccionadas
selected_features = np.array(numerical_features)[selector.get_support()]
print("Características seleccionadas:", selected_features.tolist())


Características seleccionadas: ['Elevation', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Horizontal_Distance_To_Fire_Points']


In [5]:
# Crear un nuevo DataFrame con solo las características seleccionadas
df_selected = df[selected_features.tolist()]

# Agregar la columna 'Cover_Type' de nuevo al DataFrame
df_selected[target] = df[target]

# Guardar el DataFrame en el archivo original, reemplazando el archivo CSV
df_selected.to_csv(_data_filepath, index=False)

print(f"El archivo con las características seleccionadas ha sido guardado en: {_data_filepath}")

El archivo con las características seleccionadas ha sido guardado en: /data/covertype/covertype_test.csv


/tmp/ipykernel_29/2269151857.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected[target] = df[target]


#Data Pipeline

In [6]:
#4.1 Configurar contexto interactivo

# Definir directorio de metadatos
_pipeline_root = '/data/tfx_pipeline'
_metadata_path = os.path.join(_pipeline_root, 'metadata.db')

# Crear contexto interactivo
from tfx.orchestration.metadata import sqlite_metadata_connection_config

context = InteractiveContext(
    pipeline_root=_pipeline_root,
    metadata_connection_config=sqlite_metadata_connection_config(os.path.join(_pipeline_root, 'metadata.db'))
)



In [7]:
#4.2 ExampleGen

# Crear componente ExampleGen
example_gen = CsvExampleGen(input_base=_data_root)

# Ejecutar componente
context.run(example_gen)



ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [8]:
#4.3 Estadisticas

# Crear y ejecutar el componente StatisticsGen
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
context.run(statistics_gen)

print(" StatisticsGen ejecutado correctamente.")


# Obtener la ruta del directorio de estadísticas
statistics_uri = statistics_gen.outputs['statistics'].get()[0].uri

# Rutas de los archivos FeatureStats.pb en Split-train y Split-eval
train_stats_path = os.path.join(statistics_uri, 'Split-train', 'FeatureStats.pb')
eval_stats_path = os.path.join(statistics_uri, 'Split-eval', 'FeatureStats.pb')

# Verificar si los archivos existen
if not os.path.exists(train_stats_path) or not os.path.exists(eval_stats_path):
    raise FileNotFoundError(f" No se encontraron los archivos FeatureStats.pb en {statistics_uri}")

print(f" Archivo de estadísticas de entrenamiento: {train_stats_path}")
print(f" Archivo de estadísticas de evaluación: {eval_stats_path}")

# Cargar estadísticas desde los archivos .pb
train_stats = tfdv.load_stats_binary(train_stats_path)
eval_stats = tfdv.load_stats_binary(eval_stats_path)

# Visualizar estadísticas comparando entrenamiento y evaluación
tfdv.visualize_statistics(lhs_statistics=train_stats, rhs_statistics=eval_stats, lhs_name="Train", rhs_name="Eval")


 StatisticsGen ejecutado correctamente.
 Archivo de estadísticas de entrenamiento: /data/tfx_pipeline/StatisticsGen/statistics/2/Split-train/FeatureStats.pb
 Archivo de estadísticas de evaluación: /data/tfx_pipeline/StatisticsGen/statistics/2/Split-eval/FeatureStats.pb


In [9]:
#4.4 Inferir el esquema


# Ejecutar SchemaGen
schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'])
context.run(schema_gen)

# Obtener la ruta del esquema generado
schema_uri = schema_gen.outputs['schema'].get()[0].uri
print(f" Esquema generado en: {schema_uri}")

# Verificar archivos generados
print(f" Contenido de {schema_uri}: {os.listdir(schema_uri)}")

# Cargar y visualizar el esquema
schema = tfdv.load_schema_text(os.path.join(schema_uri, 'schema.pbtxt'))
tfdv.display_schema(schema)

# Inspeccionar características inferidas
for feature in schema.feature:
    print(f" Feature: {feature.name}")
    print(f"   Tipo: {feature.type}")
    print(f"   Presencia: {feature.presence.min_fraction if feature.HasField('presence') else 'N/A'}")
    print("-" * 40)


 Esquema generado en: /data/tfx_pipeline/SchemaGen/schema/3
 Contenido de /data/tfx_pipeline/SchemaGen/schema/3: ['schema.pbtxt']


,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,-
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,-
'Hillshade_Noon',INT,required,,-
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,-
'Vertical_Distance_To_Hydrology',INT,required,,-


 Feature: Cover_Type
   Tipo: 2
   Presencia: 1.0
----------------------------------------
 Feature: Elevation
   Tipo: 2
   Presencia: 1.0
----------------------------------------
 Feature: Hillshade_9am
   Tipo: 2
   Presencia: 1.0
----------------------------------------
 Feature: Hillshade_Noon
   Tipo: 2
   Presencia: 1.0
----------------------------------------
 Feature: Horizontal_Distance_To_Fire_Points
   Tipo: 2
   Presencia: 1.0
----------------------------------------
 Feature: Horizontal_Distance_To_Hydrology
   Tipo: 2
   Presencia: 1.0
----------------------------------------
 Feature: Horizontal_Distance_To_Roadways
   Tipo: 2
   Presencia: 1.0
----------------------------------------
 Feature: Slope
   Tipo: 2
   Presencia: 1.0
----------------------------------------
 Feature: Vertical_Distance_To_Hydrology
   Tipo: 2
   Presencia: 1.0
----------------------------------------


In [10]:
#4.5 Curando el esquema


# Ejecutar SchemaGen
schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'])
context.run(schema_gen)

# Cargar el esquema inferido
schema_uri = schema_gen.outputs['schema'].get()[0].uri
schema_path = os.path.join(schema_uri, "schema.pbtxt")
schema = tfdv.load_schema_text(schema_path)

# Definir rangos permitidos para variables numéricas
tfdv.set_domain(schema, "Hillshade_9am", schema_pb2.IntDomain(min=0, max=255))
tfdv.set_domain(schema, "Hillshade_Noon", schema_pb2.IntDomain(min=0, max=255))
tfdv.set_domain(schema, "Slope", schema_pb2.IntDomain(min=0, max=90))

# Definir Cover Type como categórica
cover_type_domain = schema_pb2.IntDomain(min=0, max=6)
cover_type_feature = next(f for f in schema.feature if f.name == "Cover_Type")
cover_type_feature.int_domain.CopyFrom(cover_type_domain)
cover_type_feature.type = schema_pb2.FeatureType.INT  # Mantener como INT
cover_type_feature.annotation.tag.append("categorical")  # Marcar como categórica

# Guardar el esquema actualizado
schema_output_path = os.path.join(schema_uri, "schema.pbtxt")
tfdv.write_schema_text(schema, schema_output_path)

# Verificar cambios
schema_updated = tfdv.load_schema_text(schema_output_path)
tfdv.display_schema(schema_updated)

print(f" Esquema actualizado guardado en: {schema_output_path}")


,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,min: 0; max: 6
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,min: 0; max: 255
'Hillshade_Noon',INT,required,,min: 0; max: 255
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,min: 0; max: 90
'Vertical_Distance_To_Hydrology',INT,required,,-


 Esquema actualizado guardado en: /data/tfx_pipeline/SchemaGen/schema/3/schema.pbtxt


In [11]:
# 4.6 Entornos de Esquema

#  Directorios de datos
train_data_path = "/data/covertype/train"
inference_data_path = "/data/covertype/inference"

#  Crear directorios si no existen
os.makedirs(train_data_path, exist_ok=True)
os.makedirs(inference_data_path, exist_ok=True)

#  Cargar datos de entrenamiento y generar datos de inferencia
train_file = "/data/covertype/covertype_test.csv"
df_train = pd.read_csv(train_file)

#  Crear datos de inferencia (eliminando Cover_Type)
df_inference = df_train.drop(columns=["Cover_Type"])
df_inference.to_csv(f"{inference_data_path}/covertype_inference.csv", index=False)

#  Mover el archivo de entrenamiento a su carpeta correspondiente
df_train.to_csv(f"{train_data_path}/covertype_test.csv", index=False)

#  Cargar el esquema generado en 4.4
schema_uri = schema_gen.outputs['schema'].get()[0].uri
schema_path = os.path.join(schema_uri, "schema.pbtxt")
schema = tfdv.load_schema_text(schema_path)

#  Evitar duplicados en los entornos
if "TRAINING" not in schema.default_environment:
    schema.default_environment.append("TRAINING")
if "SERVING" not in schema.default_environment:
    schema.default_environment.append("SERVING")

# Excluir Cover_Type en SERVING sin duplicados
for feature in schema.feature:
    if feature.name == "Cover_Type" and "SERVING" not in feature.not_in_environment:
        feature.not_in_environment.append("SERVING")

# Guardar el esquema actualizado
schema_env_path = os.path.join(schema_uri, "schema.pbtxt")
tfdv.write_schema_text(schema, schema_env_path)

# Verificar configuración de entornos
print(" Entornos en el esquema:", schema.default_environment)
for feature in schema.feature:
    if feature.name == "Cover_Type":
        print(f" Feature: {feature.name}")
        print(f"   Excluido del entorno: {feature.not_in_environment}")

#  Crear ExampleGen para los datos de entrenamiento e inferencia
train_example_gen = CsvExampleGen(input_base=train_data_path)
context.run(train_example_gen)

inference_example_gen = CsvExampleGen(input_base=inference_data_path)
context.run(inference_example_gen)

#  Generar estadísticas para datos de inferencia
inference_stats_gen = StatisticsGen(examples=inference_example_gen.outputs['examples'])
context.run(inference_stats_gen)

# Validar datos de inferencia usando el esquema actualizado
example_validator = ExampleValidator(
    statistics=inference_stats_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)

context.run(example_validator)
print(" Validación de datos de inferencia completada.")


 Entornos en el esquema: ['TRAINING', 'SERVING']
 Feature: Cover_Type
   Excluido del entorno: ['SERVING']
 Validación de datos de inferencia completada.


In [12]:
#Verificación entornos


#  Cargar el esquema guardado
schema_env_path = os.path.join(schema_gen.outputs['schema'].get()[0].uri, "schema.pbtxt")
schema = tfdv.load_schema_text(schema_env_path)

#  Verificar entornos disponibles
print(" Entornos en el esquema:", schema.default_environment)

#  Verificar que Cover_Type está excluido en SERVING
for feature in schema.feature:
    if feature.name == "Cover_Type":
        print(f" Feature: {feature.name}")
        print(f"   Excluido del entorno: {feature.not_in_environment}")

#  Mostrar rangos de las columnas en Domain
print("\n Rango de valores permitidos en las características numéricas:")
for feature in schema.feature:
    if feature.HasField("int_domain"):  # Verificar si la columna tiene un dominio de valores
        print(f" {feature.name}: {feature.int_domain.min} - {feature.int_domain.max}")


 Entornos en el esquema: ['TRAINING', 'SERVING']
 Feature: Cover_Type
   Excluido del entorno: ['SERVING']

 Rango de valores permitidos en las características numéricas:
 Cover_Type: 0 - 6
 Hillshade_9am: 0 - 255
 Hillshade_Noon: 0 - 255
 Slope: 0 - 90


In [13]:
#4.7 Nuevas Estadisticas

#  Ruta del esquema actualizado
schema_env_path = os.path.join(schema_gen.outputs['schema'].get()[0].uri, "schema.pbtxt")

#  Crear componente ImportSchemaGen para importar el esquema al pipeline
schema_importer = ImportSchemaGen(schema_file=schema_env_path)

#  Ejecutar ImportSchemaGen
context.run(schema_importer)
print(" Esquema actualizado importado con ImportSchemaGen.")

#  Generar nuevas estadísticas con el esquema curado
updated_stats_gen = StatisticsGen(
    examples=example_gen.outputs['examples'],  # Usar los datos originales
    schema=schema_importer.outputs['schema']  # Pasar el esquema actualizado
)

#  Ejecutar StatisticsGen con el nuevo esquema
context.run(updated_stats_gen)
print(" Nuevas estadísticas generadas con el esquema actualizado.")

#  Ruta al archivo de estadísticas dentro de Split-train
stats_file = os.path.join(updated_stats_gen.outputs['statistics'].get()[0].uri, "Split-train", "FeatureStats.pb")

#  Cargar las estadísticas desde el archivo binario
updated_stats = tfdv.load_stats_binary(stats_file)

#  Visualizar estadísticas actualizadas
tfdv.visualize_statistics(updated_stats)


 Esquema actualizado importado con ImportSchemaGen.
 Nuevas estadísticas generadas con el esquema actualizado.


In [14]:
#4.8 Comprobar anomalias 

# Ejecutar ExampleValidator con las estadísticas y el esquema actualizados
example_validator = ExampleValidator(
    statistics=updated_stats_gen.outputs['statistics'],  # Estadísticas generadas en 4.7
    schema=schema_importer.outputs['schema']  # Esquema importado con ImportSchemaGen
)

# Ejecutar el componente para detectar anomalías
context.run(example_validator)
print("Comprobación de anomalías completada.")

# Ruta del directorio de anomalías generadas
anomalies_dir = example_validator.outputs['anomalies'].get()[0].uri
print("Ruta de salida de anomalías:", anomalies_dir)

# Verificar archivos dentro del directorio de anomalías
print("Contenido en Split-train:", os.listdir(os.path.join(anomalies_dir, "Split-train")))
print("Contenido en Split-eval:", os.listdir(os.path.join(anomalies_dir, "Split-eval")))

# Ruta al archivo de diferencias detectadas dentro de Split-train
anomalies_file = os.path.join(anomalies_dir, "Split-train", "SchemaDiff.pb")

# Verificar si el archivo de anomalías existe
if os.path.exists(anomalies_file):
    # Cargar el archivo SchemaDiff.pb como Protobuf
    anomalies_proto = anomalies_pb2.Anomalies()
    with open(anomalies_file, "rb") as f:
        anomalies_proto.ParseFromString(f.read())

    # Inspeccionar los campos del objeto anomalies_proto
    print("Estructura de anomalías (campos disponibles):")
    print(anomalies_proto)  # Esto muestra los campos disponibles en el objeto

    # Intentar ver la representación en texto del objeto completo
    print("Representación de las anomalías en formato de texto:")
    print(text_format.MessageToString(anomalies_proto))
else:
    print(f"El archivo {anomalies_file} no existe.")


Comprobación de anomalías completada.
Ruta de salida de anomalías: /data/tfx_pipeline/ExampleValidator/anomalies/11
Contenido en Split-train: ['SchemaDiff.pb']
Contenido en Split-eval: ['SchemaDiff.pb']
Estructura de anomalías (campos disponibles):
baseline {
  feature {
    name: "Cover_Type"
    type: INT
    int_domain {
      min: 0
      max: 6
    }
    presence {
      min_fraction: 1.0
      min_count: 1
    }
    annotation {
      tag: "categorical"
    }
    not_in_environment: "SERVING"
    shape {
      dim {
        size: 1
      }
    }
  }
  feature {
    name: "Elevation"
    type: INT
    presence {
      min_fraction: 1.0
      min_count: 1
    }
    shape {
      dim {
        size: 1
      }
    }
  }
  feature {
    name: "Hillshade_9am"
    type: INT
    int_domain {
      min: 0
      max: 255
    }
    presence {
      min_fraction: 1.0
      min_count: 1
    }
    shape {
      dim {
        size: 1
      }
    }
  }
  feature {
    name: "Hillshade_Noon"
    

#Ingeneria de Caracteristicas - Transform

In [15]:
# Crear el componente Transform
transform = Transform(
    examples=example_gen.outputs["examples"],  # Datos de entrada (del paso ExampleGen)
    schema=schema_importer.outputs["schema"],  # Esquema curado (del paso SchemaGen o ImportSchemaGen)
    module_file="/tfx/notebooks/transform_module.py"  # Archivo con la función de transformación
)

# Ejecutar el componente Transform
context.run(transform)

# Obtener la salida de las características transformadas
transform_graph_uri = transform.outputs["transform_graph"].get()[0].uri
transformed_examples_uri = transform.outputs["transformed_examples"].get()[0].uri

print(f"Transformaciones completadas. Gráfica de transformación guardada en: {transform_graph_uri}")
print(f"Datos transformados guardados en: {transformed_examples_uri}")

# Verificar archivos generados en el directorio de datos transformados
tfrecord_files = tf.io.gfile.glob(os.path.join(transformed_examples_uri, "Split-train/*"))

# Asegurar que se generaron archivos correctamente
if not tfrecord_files:
    raise FileNotFoundError(f"No se encontraron archivos TFRecord en {transformed_examples_uri}/Split-train")

# Manejo de archivos comprimidos
dataset = tf.data.TFRecordDataset(tfrecord_files, compression_type="GZIP")

# Leer algunos ejemplos transformados para verificar los cambios
print("\nEjemplos transformados (primeros 5 registros):")
for raw_record in dataset.take(5):
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    print(example)


running bdist_wheel
running build
running build_py
creating build/lib
copying transform_module.py -> build/lib
installing to /tmp/tmp96hd60ls
running install
running install_lib
copying build/lib/transform_module.py -> /tmp/tmp96hd60ls/.
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmp96hd60ls/./tfx_user_code_Transform-0.0+e987611f39fc3b22a4be26815b5df591d8a4f7b1a87aead90e54c89fc8713871-py3.10.egg-info
running install_scripts
creating /tmp/tmp96hd60ls/tfx_user_code_transform-0.0+e987611f39fc3b2

/opt/conda/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:79: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


Processing /data/tfx_pipeline/_wheels/tfx_user_code_transform-0.0+e987611f39fc3b22a4be26815b5df591d8a4f7b1a87aead90e54c89fc8713871-py3-none-any.whl


Processing /data/tfx_pipeline/_wheels/tfx_user_code_transform-0.0+e987611f39fc3b22a4be26815b5df591d8a4f7b1a87aead90e54c89fc8713871-py3-none-any.whl


Processing /data/tfx_pipeline/_wheels/tfx_user_code_transform-0.0+e987611f39fc3b22a4be26815b5df591d8a4f7b1a87aead90e54c89fc8713871-py3-none-any.whl


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


Transformaciones completadas. Gráfica de transformación guardada en: /data/tfx_pipeline/Transform/transform_graph/12
Datos transformados guardados en: /data/tfx_pipeline/Transform/transformed_examples/12

Ejemplos transformados (primeros 5 registros):
features {
  feature {
    key: "Cover_Type"
    value {
      int64_list {
        value: 1
      }
    }
  }
  feature {
    key: "Elevation_minmax"
    value {
      float_list {
        value: 0.5686274766921997
      }
    }
  }
  feature {
    key: "Hillshade_9am_minmax"
    value {
      float_list {
        value: 0.9173228144645691
      }
    }
  }
  feature {
    key: "Hillshade_Noon_minmax"
    value {
      float_list {
        value: 0.9212598204612732
      }
    }
  }
  feature {
    key: "Horizontal_Distance_To_Fire_Points_minmax"
    value {
      float_list {
        value: 0.2190290242433548
      }
    }
  }
  feature {
    key: "Horizontal_Distance_To_Hydrology_minmax"
    value {
      float_list {
        value: 0.

2025-03-01 23:25:16.161673: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


#Metadatos de aprendizaje automático

In [22]:
# Usar la misma ruta de metadatos que en el pipeline
METADATA_PATH = "/data/tfx_pipeline/metadata.db"  # Asegurar que es metadata.db y no metadata.sqlite

# Crear una conexión con el almacén de metadatos
metadata_config = sqlite_metadata_connection_config(METADATA_PATH)
store = MetadataStore(metadata_config)

print(f" Conexión establecida con el almacén de metadatos en {METADATA_PATH}")


 Conexión establecida con el almacén de metadatos en /data/tfx_pipeline/metadata.db


In [30]:
def listar_tipos_de_artefactos():
    """Lista todos los tipos de artefactos almacenados en ML Metadata con sus IDs."""
    artifact_types = store.get_artifact_types()
    
    print("\n Tipos de artefactos almacenados en el pipeline:")
    for artifact in artifact_types:
        print(f"- ID: {artifact.id}, Nombre: {artifact.name}")

# Ejecutar la función
listar_tipos_de_artefactos()



 Tipos de artefactos almacenados en el pipeline:
- ID: 14, Nombre: Examples
- ID: 16, Nombre: ExampleStatistics
- ID: 18, Nombre: Schema
- ID: 20, Nombre: ExampleAnomalies
- ID: 23, Nombre: TransformGraph
- ID: 24, Nombre: TransformCache


In [25]:
def obtener_artefactos_por_tipo(tipo_artefacto):
    """Lista los artefactos de un tipo específico y sus propiedades."""
    artifacts = store.get_artifacts_by_type(tipo_artefacto)

    print(f"\n Artefactos de tipo: {tipo_artefacto}")
    for artifact in artifacts:
        # Obtener eventos asociados al artefacto
        events = store.get_events_by_artifact_ids([artifact.id])
        
        # Filtrar eventos de salida (que indican qué componente generó el artefacto)
        producer_execution_id = None
        for event in events:
            if event.type == metadata_store_pb2.Event.OUTPUT:
                producer_execution_id = event.execution_id
                break  # Tomamos solo el primer evento de salida

        # Obtener el nombre del componente productor
        producer_component = "Desconocido"
        if producer_execution_id:
            execution = store.get_executions_by_id([producer_execution_id])[0]
            producer_component = execution.properties["pipeline_step"].string_value

        print(f"- ID: {artifact.id}, URI: {artifact.uri}, Estado: {artifact.state}, Creado por: {producer_component}")

# Obtener información sobre los esquemas almacenados
obtener_artefactos_por_tipo("Schema")



 Artefactos de tipo: Schema
- ID: 3, URI: /data/tfx_pipeline/SchemaGen/schema/3, Estado: 2, Creado por: 
- ID: 8, URI: /data/tfx_pipeline/ImportSchemaGen/schema/9, Estado: 2, Creado por: 
- ID: 14, URI: /data/tfx_pipeline/Transform/pre_transform_schema/12, Estado: 2, Creado por: 
- ID: 16, URI: /data/tfx_pipeline/Transform/post_transform_schema/12, Estado: 2, Creado por: 


In [27]:
def obtener_propiedades_artefacto(artifact_id):
    """Muestra todas las propiedades de un artefacto en particular, incluyendo el componente que lo generó."""
    artifact = store.get_artifacts_by_id([artifact_id])[0]

    print(f"\n Propiedades del artefacto ID {artifact_id}:")
    print(f"- Tipo ID: {artifact.type_id}")
    print(f"- URI: {artifact.uri}")
    print(f"- Estado: {artifact.state}")

    # Obtener eventos asociados al artefacto
    events = store.get_events_by_artifact_ids([artifact.id])
    
    # Filtrar eventos de salida (que indican qué componente generó el artefacto)
    producer_execution_id = None
    for event in events:
        if event.type == metadata_store_pb2.Event.OUTPUT:
            producer_execution_id = event.execution_id
            break  # Tomamos solo el primer evento de salida

    # Obtener el nombre del componente productor
    producer_component = "Desconocido"
    if producer_execution_id:
        execution = store.get_executions_by_id([producer_execution_id])[0]
        producer_component = execution.properties["pipeline_step"].string_value

    print(f"- Creado por: {producer_component}")

    # Mostrar propiedades adicionales
    for key, value in artifact.custom_properties.items():
        print(f"- {key}: {value}")

# Obtener detalles de un Schema específico (por ejemplo, ID=5)
obtener_propiedades_artefacto(5)



 Propiedades del artefacto ID 5:
- Tipo ID: 14
- URI: /data/tfx_pipeline/CsvExampleGen/examples/6
- Estado: 2
- Creado por: 
- input_fingerprint: string_value: "split:single_split,num_files:1,total_bytes:3712931,xor_checksum:1740871436,sum_checksum:1740871436"

- file_format: string_value: "tfrecords_gzip"

- span: int_value: 0

- payload_format: string_value: "FORMAT_TF_EXAMPLE"

- tfx_version: string_value: "1.16.0"



In [28]:
def obtener_artefactos_entrada(artifact_ids):
    """Devuelve los artefactos de entrada utilizados para generar una lista de artefactos."""

    if not isinstance(artifact_ids, list):
        artifact_ids = [artifact_ids]  # Convertir en lista si es un solo ID

    artifact_list = store.get_artifacts_by_id(artifact_ids)

    if not artifact_list:
        print(f"No se encontraron artefactos con los IDs {artifact_ids}")
        return

    for artifact in artifact_list:
        print(f"\n Buscando artefactos de entrada para: Tipo {artifact.type_id} (ID {artifact.id})")
        print(f"URI: {artifact.uri}")

        # Obtener eventos en los que este artefacto fue una salida
        events = store.get_events_by_artifact_ids([artifact.id])

        # Obtener las ejecuciones que generaron este artefacto
        executions = [
            event.execution_id for event in events if event.type == metadata_store_pb2.Event.OUTPUT
        ]

        if not executions:
            print(" No se encontraron ejecuciones asociadas a este artefacto.")
            continue

        print(f" Generado en las ejecuciones: {executions}")

        # Obtener eventos de entrada en esas ejecuciones
        execution_events = store.get_events_by_execution_ids(executions)

        # Filtrar eventos que representen entradas
        input_artifacts = list({
            event.artifact_id for event in execution_events if event.type == metadata_store_pb2.Event.INPUT
        })  # Convertir a lista para evitar duplicados

        if not input_artifacts:
            print("No se encontraron artefactos de entrada para este artefacto.")
            continue

        print("\n Artefactos utilizados como entrada:")
        for input_artifact_id in input_artifacts:
            input_artifact_list = store.get_artifacts_by_id([input_artifact_id])
            if input_artifact_list:
                input_artifact = input_artifact_list[0]
                print(f"- ID: {input_artifact.id}, Tipo: {input_artifact.type_id}, URI: {input_artifact.uri}")



In [29]:
# Obtener artefactos de entrada para múltiples artefactos
obtener_artefactos_entrada([2, 3, 5])



 Buscando artefactos de entrada para: Tipo 16 (ID 2)
URI: /data/tfx_pipeline/StatisticsGen/statistics/2
 Generado en las ejecuciones: [2]

 Artefactos utilizados como entrada:
- ID: 1, Tipo: 14, URI: /data/tfx_pipeline/CsvExampleGen/examples/1

 Buscando artefactos de entrada para: Tipo 18 (ID 3)
URI: /data/tfx_pipeline/SchemaGen/schema/3
 Generado en las ejecuciones: [3, 4]

 Artefactos utilizados como entrada:
- ID: 2, Tipo: 16, URI: /data/tfx_pipeline/StatisticsGen/statistics/2

 Buscando artefactos de entrada para: Tipo 14 (ID 5)
URI: /data/tfx_pipeline/CsvExampleGen/examples/6
 Generado en las ejecuciones: [6]
No se encontraron artefactos de entrada para este artefacto.
